In [ ]:
%run main.py
%load_ext autoreload
%autoreload 2

!mkdir -p {DATA_DIR} {BERT_DIR} {MODEL_DIR}
s3 = S3()

In [ ]:
if not exists(NEWS):
    s3.download(S3_NEWS, NEWS)
    s3.download(S3_WIKI, WIKI)
    s3.download(S3_FICTION, FICTION)

In [ ]:
if not exists(BERT_VOCAB):
    s3.download(S3_BERT_VOCAB, BERT_VOCAB)
    s3.download(S3_BERT_EMB, BERT_EMB)
    s3.download(S3_BERT_ENCODER, BERT_ENCODER)

In [ ]:
items = list(load_lines(BERT_VOCAB))
words_vocab = BERTVocab(items)

In [ ]:
markups = {}
for path, name in [(NEWS, TEST), (FICTION, TRAIN)]:
    lines = load_gz_lines(path)
    items = parse_jl(lines)
    items = log_progress(items, desc=path)
    records = []
    for item in items:
        record = MorphMarkup.from_json(item)
        records.append(record)
    markups[name] = records

tags = set()
for name in [TEST, TRAIN]:
    for markup in markups[name]:
        for token in markup.tokens:
            tags.add(token.tag)
            
tags = [PAD] + sorted(tags)
tags_vocab = TagsVocab(tags)

In [ ]:
torch.manual_seed(SEED)
seed(SEED)

In [ ]:
config = RuBERTConfig()
emb = BERTEmbedding(
    config.vocab_size, config.seq_len, config.emb_dim,
    config.dropout, config.norm_eps
)
encoder = BERTEncoder(
    config.layers_num, config.emb_dim, config.heads_num, config.hidden_dim,
    config.dropout, config.norm_eps
)
morph = BERTMorphHead(config.emb_dim, len(tags_vocab))
model = BERTMorph(emb, encoder, morph)

for param in emb.parameters():
    param.requires_grad = False

load_model(model.emb, BERT_EMB)
load_model(model.encoder, BERT_ENCODER)
model = model.to(DEVICE)

criterion = masked_flatten_cross_entropy

In [ ]:
encode = BERTMorphEncoder(
    words_vocab, tags_vocab,
    seq_len=128,
    batch_size=32,
    shuffle_size=10000
)

batches = {}
for name in [TEST, TRAIN]:
    batches[name] = [_.to(DEVICE) for _ in encode(markups[name])]

In [ ]:
board = Board(BOARD_NAME, RUNS_DIR)
boards = {
    TRAIN: board.section(TRAIN_BOARD),
    TEST: board.section(TEST_BOARD),
}

In [ ]:
optimizer = optim.Adam([
    dict(params=encoder.parameters(), lr=BERT_LR),
    dict(params=morph.parameters(), lr=LR),
])
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, LR_GAMMA)

In [ ]:
meters = {
    TRAIN: MorphScoreMeter(),
    TEST: MorphScoreMeter()
}

for epoch in log_progress(range(EPOCHS)):
    model.train()
    for batch in log_progress(batches[TRAIN], leave=False):
        optimizer.zero_grad()
        batch = process_batch(model, criterion, batch)
        batch.loss.backward()
        optimizer.step()
    
        score = score_morph_batch(batch)
        meters[TRAIN].add(score)

    meters[TRAIN].write(boards[TRAIN])
    meters[TRAIN].reset()

    model.eval()
    with torch.no_grad():
        for batch in log_progress(batches[TEST], leave=False, desc=TEST):
            batch = process_morph_batch(model, criterion, batch)
            score = score_batch(batch)
            meters[TEST].add(score)
        meters[TEST].write(boards[TEST])
        meters[TEST].reset()
    
    scheduler.step()
    board.step()

In [ ]:
dump_model(model.encoder, MODEL_ENCODER)
dump_model(model.morph, MODEL_MORPH)
dump_lines(tags_vocab.items, TAGS_VOCAB)
        
s3.upload(MODEL_ENCODER, S3_MODEL_ENCODER)
s3.upload(MODEL_MORPH, S3_MODEL_MORPH)
s3.upload(TAGS_VOCAB, S3_TAGS_VOCAB)